In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["KERAS_BACKEND"] = "torch"  # or "tensorflow" or "torch"
# os.environ["WANDB_SILENT"] = "false" # for wandb

import keras_nlp
import keras_core as keras 
import keras_core.backend as K


# import torch
# import jax
import tensorflow as tf
# from tensorflow import keras
# import tensorflow.keras.backend as K

import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split

Using PyTorch backend.


2023-08-19 21:40:50.818803: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 21:40:51.609217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   try:
#     # Currently, memory growth needs to be the same across GPUs
#     for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Memory growth must be set before GPUs have been initialized
#     print(e)

In [3]:
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("KerasNLP:", keras_nlp.__version__)

TensorFlow: 2.12.0
Keras: 0.1.4
KerasNLP: 0.6.1


In [4]:
class CFG:
    verbose = 0  # Verbosity
    wandb = True  # Weights & Biases logging
    competition = 'kaggle-llm-science-exam'  # Competition name
    
    preset = "deberta_v3_base_en"  # Name of pretrained models
    sequence_length = 200  # Input sequence length
    
    device = 'GPU'  # Device
    
    seed = 42  # Random seed
    
    selected_folds = [1, 2]  # Folds to train on
    
    epochs = 2  # Training epochs
    batch_size = 2  # Batch size
    drop_remainder = True  # Drop incomplete batches
    cache = True # Caches data after one iteration, use only with `TPU` to avoid OOM
    
    augment = True # Augmentation (Shuffle Options)
    
    scheduler = 'cosine'  # Learning rate scheduler
    
    external_data = True  # External data flag
    
    class_names = list("ABCDE")  # Class names [A, B, C, D, E]
    num_classes = len(class_names)  # Number of classes
    class_labels = list(range(num_classes))  # Class labels [0, 1, 2, 3, 4]
    label2name = dict(zip(class_labels, class_names))  # Label to class name mapping
    name2label = {v: k for k, v in label2name.items()}  # Class name to label mapping

In [5]:
def get_device():
    "Detect and intializes GPU/TPU automatically"
    try:
        # Connect to TPU
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() 
        # Set TPU strategy
        strategy = tf.distribute.TPUStrategy(tpu)
        print(f'> Running on TPU', tpu.master(), end=' | ')
        print('Num of TPUs: ', strategy.num_replicas_in_sync)
        device=CFG.device
    except:
        # If TPU is not available, detect GPUs
        gpus = tf.config.list_logical_devices('GPU')
        ngpu = len(gpus)
         # Check number of GPUs
        if ngpu:
            # Set GPU strategy
            strategy = tf.distribute.MirroredStrategy(gpus) # single-GPU or multi-GPU
            # Print GPU details
            print("> Running on GPU", end=' | ')
            print("Num of GPUs: ", ngpu)
            device='GPU'
        else:
            # If no GPUs are available, use CPU
            print("> Running on CPU")
            strategy = tf.distribute.get_strategy()
            device='CPU'
    return strategy, device


strategy, CFG.device = get_device()
CFG.replicas = strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
> Running on GPU | Num of GPUs:  1


2023-08-19 21:27:30.305976: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-19 21:27:30.307033: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-19 21:27:30.307225: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
df = pd.read_csv("../data/train_extra_data.csv")
df.head()

,prompt,A,B,C,D,E,answer
0,Who is the dean of the University of Central F...,John Doe,Michael Georgiopoulos,Jane Smith,Robert Johnson,David Brown,B
1,What degrees does the college offer?,Engineering and computer science,Business and economics,Psychology and sociology,Mathematics and statistics,Arts and humanities,A
2,How many undergraduate and graduate students w...,"Over 5,000","Over 10,000","Over 7,500","Over 3,000","Over 15,000",C
3,What is the University of Central Florida list...,A liberal arts college,A research university,An Ivy League institution,A community college,A vocational school,B
4,What are some of the research contributions ma...,Architecture and design,Medicine and healthcare,Environmental science,Modeling and simulation,Linguistics and literature,D


> We will use the original train.csv data provided by the hosts as the evaluation set to compare models.

In [6]:
def add_label(row):
    if row.answer == "A":
        return 0
    elif row.answer == "B":
        return 1
    elif row.answer == "C":
        return 2
    elif row.answer == "D":
        return 3
    else:
        return 4

    return


def make_options(row):
    return [f"{row.prompt}\n{row.A}",  # Option A
            f"{row.prompt}\n{row.B}",  # Option B
            f"{row.prompt}\n{row.C}",  # Option C
            f"{row.prompt}\n{row.D}",  # Option D
            f"{row.prompt}\n{row.E}"]  # Option E

In [7]:
df["label"] = df.apply(lambda row: add_label(row), axis=1)
df["options"] = df.apply(lambda row: make_options(row), axis=1)

In [8]:
df.head()

,prompt,A,B,C,D,E,answer,label,options
0,Who is the dean of the University of Central F...,John Doe,Michael Georgiopoulos,Jane Smith,Robert Johnson,David Brown,B,1,[Who is the dean of the University of Central ...
1,What degrees does the college offer?,Engineering and computer science,Business and economics,Psychology and sociology,Mathematics and statistics,Arts and humanities,A,0,[What degrees does the college offer?\nEnginee...
2,How many undergraduate and graduate students w...,"Over 5,000","Over 10,000","Over 7,500","Over 3,000","Over 15,000",C,2,[How many undergraduate and graduate students ...
3,What is the University of Central Florida list...,A liberal arts college,A research university,An Ivy League institution,A community college,A vocational school,B,1,[What is the University of Central Florida lis...
4,What are some of the research contributions ma...,Architecture and design,Medicine and healthcare,Environmental science,Modeling and simulation,Linguistics and literature,D,3,[What are some of the research contributions m...


In [9]:
train_df, valid_df = train_test_split(df, test_size=0.10, random_state=42, stratify=list(df.label.values))
print(len(train_df), len(valid_df))

9675 1076


## Preprocessor

In [10]:
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    preset=CFG.preset, # Name of the model
    sequence_length=CFG.sequence_length, # Max sequence length, will be padded if shorter
)

2023-08-19 21:41:05.973579: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-19 21:41:05.974737: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-19 21:41:05.974970: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## Dataloader

In [11]:
AUTO = tf.data.AUTOTUNE

In [12]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (
        list(train_df["options"].values),
        list(train_df["label"].values)
    )
)

    
valid_ds = tf.data.Dataset.from_tensor_slices(
    (
        list(valid_df["options"].values),
        list(valid_df["label"].values)
    )
)

In [13]:
next(iter(train_ds))

2023-08-19 21:41:14.648475: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [9675]
	 [[{{node Placeholder/_1}}]]


(<tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b'What types of notifications can be pushed by a Push Proxy Gateway?\nMMS, email, IM',
        b'What types of notifications can be pushed by a Push Proxy Gateway?\nRingtone downloads, new device firmware notifications',
        b'What types of notifications can be pushed by a Push Proxy Gateway?\nMMS, email, IM, ringtone downloads, new device firmware notifications',
        b'What types of notifications can be pushed by a Push Proxy Gateway?\nEmail, IM, ringtone downloads, new device firmware notifications',
        b'What types of notifications can be pushed by a Push Proxy Gateway?\nMMS, IM, ringtone downloads'],
       dtype=object)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=2>)

In [14]:
def preprocess_fn(text, label):
    text = preprocessor(text)

    label = tf.one_hot(label, depth=5)
    return text, label

In [15]:
trainloader = (
    train_ds
    .shuffle(1024)
    .map(preprocess_fn, num_parallel_calls=2)
    .batch(CFG.batch_size, drop_remainder=True)
    .prefetch(2)
)


validloader = (
    valid_ds
    .map(preprocess_fn, num_parallel_calls=2)
    .batch(CFG.batch_size, drop_remainder=True)
    .prefetch(2)
)

2023-08-19 21:41:20.590586: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Trimmer/Trim/RoundRobinTrimmer/generate_mask/while/Tile/multiples/Trimmer/Trim/RoundRobinTrimmer/generate_mask/strided_slice' with dtype int32
	 [[{{node Trimmer/Trim/RoundRobinTrimmer/generate_mask/while/Tile/multiples/Trimmer/Trim/RoundRobinTrimmer/generate_mask/strided_slice}}]]
2023-08-19 21:41:20.594583: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Trimmer/Trim/RoundRobinTrimmer/generate_mask/while/Tile/multiples/Trimmer/Trim/RoundRobinTrimmer/generate_mask/strided_slice' with dtype int32
	 [[{{node Trimmer/Trim/RoundRobinTrimmer/genera

In [16]:
next(iter(trainloader))

2023-08-19 21:41:25.641694: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [9675,5]
	 [[{{node Placeholder/_0}}]]
2023-08-19 21:41:25.642137: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [9675]
	 [[{{node Placeholder/_1}}]]
2023-08-19 21:41:25.906872: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): CANCELLED: Loop execution was cancelled.
	 [[{{node Trimmer/Trim/RoundRobinTrimme

({'token_ids': <tf.Tensor: shape=(2, 5, 200), dtype=int32, numpy=
  array([[[  1, 577, 464, ...,   0,   0,   0],
          [  1, 577, 464, ...,   0,   0,   0],
          [  1, 577, 464, ...,   0,   0,   0],
          [  1, 577, 464, ...,   0,   0,   0],
          [  1, 577, 464, ...,   0,   0,   0]],
  
         [[  1, 458, 269, ...,   0,   0,   0],
          [  1, 458, 269, ...,   0,   0,   0],
          [  1, 458, 269, ...,   0,   0,   0],
          [  1, 458, 269, ...,   0,   0,   0],
          [  1, 458, 269, ...,   0,   0,   0]]], dtype=int32)>,
  'padding_mask': <tf.Tensor: shape=(2, 5, 200), dtype=bool, numpy=
  array([[[ True,  True,  True, ..., False, False, False],
          [ True,  True,  True, ..., False, False, False],
          [ True,  True,  True, ..., False, False, False],
          [ True,  True,  True, ..., False, False, False],
          [ True,  True,  True, ..., False, False, False]],
  
         [[ True,  True,  True, ..., False, False, False],
          [ True,

## Model

In [17]:
# Selects one option from five
class SelectOption(keras.layers.Layer):
    def __init__(self, index, **kwargs):
        super().__init__(**kwargs)
        self.index = index

    def call(self, inputs):
        # Selects a specific slice from the inputs tensor
        return inputs[:, self.index, :]
    
    def get_config(self):
        # For serialize the model
        base_config = super().get_config()
        config = {
            "index": self.index,
        }
        return {**base_config, **config}

In [18]:
def build_model():
    # Define input layers
    inputs = {
        "token_ids": keras.Input(shape=(5, None), dtype=tf.int32, name="token_ids"),
        "padding_mask": keras.Input(shape=(5, None), dtype=tf.int32, name="padding_mask"),
    }
    # Create a DebertaV3Classifier model
    classifier = keras_nlp.models.DebertaV3Classifier.from_preset(
        CFG.preset,
        preprocessor=None,
        num_classes=1 # one output per one option, for five options total 5 outputs
    )
    logits = []
    # Loop through each option (Q+A), (Q+B) etc and compute associted logits
    for option_idx in range(5):
        option = {k: SelectOption(option_idx, name=f"{k}_{option_idx}")(v) for k, v in inputs.items()}
        logit = classifier(option)
        logits.append(logit)

    # Compute final output
    logits = keras.layers.Concatenate(axis=-1)(logits)
    outputs = keras.layers.Softmax(axis=-1)(logits)
    model = keras.Model(inputs, outputs)
    
    return model

In [19]:
tf.keras.backend.clear_session()
model = build_model()
model.summary()

/opt/conda/envs/llm-exam/lib/python3.10/site-packages/keras_core/src/trainers/trainer.py:66: UserWarning: `jit_compile` is not yet enabled for the PyTorch backend. Proceeding with `jit_compile=False`.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask        │ (None, 5, None)   │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ token_ids           │ (None, 5, None)   │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ padding_mask_0      │ (None, None)      │       0 │ padding_mask[0][0]   │
│ (SelectOption)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ token_ids_0         │ (None, None)      │       0 │ token_ids[0][0]      │
│ (SelectOption)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ padding_mask_1      │ (None, None)      │       0 │ padding_mask[0][0]   │
│ (SelectOption)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ token_ids_1         │ (None, None)      │       0 │ token_ids[0][0]      │
│ (SelectOption)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ padding_mask_2      │ (None, None)      │       0 │ padding_mask[0][0]   │
│ (SelectOption)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ token_ids_2         │ (None, None)      │       0 │ token_ids[0][0]      │
│ (SelectOption)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ padding_mask_3      │ (None, None)      │       0 │ padding_mask[0][0]   │
│ (SelectOption)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ token_ids_3         │ (None, None)      │       0 │ token_ids[0][0]      │
│ (SelectOption)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ padding_mask_4      │ (None, None)      │       0 │ padding_mask[0][0]   │
│ (SelectOption)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ token_ids_4         │ (None, None)      │       0 │ token_ids[0][0]      │
│ (SelectOption)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ deberta_v3_classif… │ (None, 1)         │ 184,42… │ padding_mask_0[0][0… │
│ (DebertaV3Classifi… │                   │         │ token_ids_0[0][0],   │
│                     │                   │         │ padding_mask_1[0][0… │
│                     │                   │         │ token_ids_1[0][0],   │
│                     │                   │         │ padding_mask_2[0][0… │
│                     │                   │         │ token_ids_2[0][0],   │
│                     │                   │         │ padding_mask_3[0][0… │
│                     │                   │         │ token_ids_3[0][0],   │
│                     │                   │         │ padding_mask_4[0][0… │
│                     │                   │         │ token_ids_4[0][0]    │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ concatenate         │ (None, 5)         │       0 │ deberta_v3_classifi… │
│ (Concatenate)       │                   │         │ deberta_v3_classifi

 Total params: 184,422,913 (5.50 GB)

 Trainable params: 184,422,913 (5.50 GB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Compile the model with optimizer, loss, and metrics
model.compile(
    optimizer=keras.optimizers.AdamW(5e-6),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.02),
    metrics=[
        keras.metrics.CategoricalAccuracy(name="accuracy"),
        keras.metrics.TopKCategoricalAccuracy(k=3, name="accuracy@3")
    ],
    jit_compile=False
)

## Train

In [21]:
model.fit(
    trainloader,
    epochs=CFG.epochs,
    validation_data=validloader,
)

2023-08-19 21:41:42.638179: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at sentencepiece_kernels.cc:275 : NOT_FOUND: Resource localhost/_0_SentencepieceOp/N10tensorflow4text12_GLOBAL__N_121SentencepieceResourceE does not exist.
2023-08-19 21:41:42.638253: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): NOT_FOUND: Resource localhost/_0_SentencepieceOp/N10tensorflow4text12_GLOBAL__N_121SentencepieceResourceE does not exist.
	 [[{{node SentenceTokenizer/SentencepieceTokenizeOp}}]]
2023-08-19 21:41:42.638265: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at sentencepiece_kernels.cc:275 : NOT_FOUND: Resource localhost/_0_SentencepieceOp/N10tensorflow4text12_GLOBAL__N_121SentencepieceResourceE does not exist.
2023-08-19 21:41:42.638311: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localh

NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} Resource localhost/_0_SentencepieceOp/N10tensorflow4text12_GLOBAL__N_121SentencepieceResourceE does not exist.
	 [[{{node SentenceTokenizer/SentencepieceTokenizeOp}}]] [Op:IteratorGetNext]